### Challenges This Week

* With `Surprise` library, processing time too long on the large dataset
* Tried `Pyspark`, to take advantage of it's distributed processing:
    - mapping `str` type userIDs to `int` type userIDs was necessary
    - processing time for training/evaluating extremely long due to ALS algo --> Tried using a smaller subset of data, still too long
    - Terrible RMSE after evaluating baseline model (RMSE: 4.719, peviously with `Surprise`, it was 0.99-1.1)
    - pyspark session abruptly stopping due to running out of memory (?) during cross-val, even with smaller subset of data

### Next Steps

* Revert back to `Surprise` library for modeling since no need to map `str` type userIDs to `int` type, and more options for algorithms
* Most likely will have to only use a fraction of the overall dataset instead of entire dataset due to processing time


**Note**: Dataset contained 51M+ rows, and 46M+ rows after dropping rows containing `None` type values.



<hr>

Import libraries

In [1]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, IntegerType, FloatType, StringType, LongType
from pyspark.sql.functions import col

from surprise import Reader, Dataset, SVD, accuracy
from surprise.model_selection import train_test_split, cross_validate, GridSearchCV
from surprise import KNNBasic, KNNBaseline, KNNWithMeans

Create SparkSession

In [2]:
spark = SparkSession.builder.master("local[6]").appName('Book Ratings')\
                            .config('spark.executor.memory', '8g')\
                            .config('spark.driver.memory', '4g')\
                            .getOrCreate()
spark

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/09 22:09:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Import Dataset into Spark DF

In [3]:
# Get underlying Spark Context
sc = spark.sparkContext

In [4]:
# define new schema
schema = StructType()\
        .add('bookID', IntegerType(), nullable=False)\
        .add('userID', StringType(), nullable=False)\
        .add('rating', FloatType(), nullable=False)\
        .add('timestamp', LongType(), nullable=False)

In [5]:
# Import data into PySpark DF
books = spark.read.format('csv').schema(schema).load('../data/Books.csv')
books.head()

Row(bookID=1713353, userID='A1C6M8LCIX4M6M', rating=5.0, timestamp=1123804800)

In [6]:
# Verify data types
books.printSchema()

root
 |-- bookID: integer (nullable = true)
 |-- userID: string (nullable = true)
 |-- rating: float (nullable = true)
 |-- timestamp: long (nullable = true)



The data is returned in the format `(int(bookID), str(userID), float(rating), long(timestamp))`.

To parse into a `PySpark` `Rating` object, it is expected to be in the format `(int(userID), int(bookID), float(rating), long(timestamp))`.

<hr>

### Check for `NoneType` Values

In [7]:
# Check for None values in the DF
none_count = books.filter(
    (col("bookID").isNull()) |
    (col("userID").isNull()) |
    (col("rating").isNull()) |
    (col("timestamp").isNull())
).count()
has_none_values = none_count > 0

# Print the result
if has_none_values:
    print(f"The DF has at least {none_count} rows with a None value.")
else:
    print("The DF does not have any rows with None values.")

The DF has at least 6671993 rows with a None value.


In [8]:
# Verify what rows with None values look like
none_df = books.filter(
    (col('bookID').isNull()) |
    (col('userID').isNull()) |
    (col('rating').isNull()) |
    (col('timestamp').isNull())
)
none_df.take(10)

[Row(bookID=None, userID='ASS457AQPDIFZ', rating=5.0, timestamp=1409443200),
 Row(bookID=None, userID='A3NMH1KTLG7CWX', rating=5.0, timestamp=1398816000),
 Row(bookID=None, userID='A2LI5026JCXQBA', rating=4.0, timestamp=1398729600),
 Row(bookID=None, userID='AHNMXYVRDN1R9', rating=5.0, timestamp=1394323200),
 Row(bookID=None, userID='A2CAVTNQA2Y3IJ', rating=5.0, timestamp=1384560000),
 Row(bookID=None, userID='A2685NTFXLJJ1T', rating=5.0, timestamp=1377475200),
 Row(bookID=None, userID='A17TBLPM7H401J', rating=4.0, timestamp=1374364800),
 Row(bookID=None, userID='A1840OJGNFSBSN', rating=5.0, timestamp=900460800),
 Row(bookID=None, userID='A3ONKN7GMHG6K2', rating=5.0, timestamp=1400630400),
 Row(bookID=None, userID='A4LSI6PTX23BE', rating=5.0, timestamp=1400284800)]

The number of rows with missing values represent about 13% of the entire dataset.   
Since there isn't a straightforward way of handling the missing values without affecting the results of the recommendations, the rows will simply be dropped.

In [9]:
# Drop missing values
books = books.dropna()

# Verify count
books.count()

44639628

After dropping rows with missing values, we still have around ~44.6M rows of data left to work with.

<hr>

### Subset Dataset based on Books Titles
Due to the large size of the dataset and processing issues, the dataset will be downsampled to about 1,000,000 rows based on books.

**Rationale**:
Since each book on average received about 17 distinct ratings, downsampling the dataset to about 1M rows will require about 58,000 unique books.  
This is still a very large dataset and should be able to return decent results for our recommendation engine.

Convert DF to RDD for Mapping UserIDs

In [10]:
# Convert DF to RDD 
books_rdd = books.rdd.map(list)
books_rdd.first()

[1713353, 'A1C6M8LCIX4M6M', 5.0, 1123804800]

In [11]:
# Get all book IDs
book_ids = books_rdd.map(lambda x: x[0])

# Verify result
book_ids.first()



1713353

Get all unique bookIDs for mapping.

In [12]:
# Get all unique book IDs
book_ids = book_ids.distinct()

# Count total book titles 
total_books = book_ids.count()

print(f"There are {total_books} total books in the dataset.")

Java HotSpot(TM) 64-Bit Server VM warning: CodeCache is full. Compiler has been disabled.
Java HotSpot(TM) 64-Bit Server VM warning: Try increasing the code cache size using -XX:ReservedCodeCacheSize=


CodeCache: size=131072Kb used=22607Kb max_used=22617Kb free=108464Kb
 bounds [0x00000001089e0000, 0x000000010a020000, 0x00000001109e0000]
 total_blobs=9396 nmethods=8446 adapters=861
 compilation: disabled (not enough contiguous free space left)


There are 2266596 total books in the dataset.


In [13]:
# Get a Random Sample of 58,000 books from list of unique books IDs
book_ids_58k = book_ids.sample(withReplacement=False, fraction=0.026, seed=42)

In [14]:
book_ids_58k.count()

58851

In [15]:
# Broadcast values to nodes and perform collect transformation to be used in filter
broadcasted_book_ids = sc.broadcast(set(book_ids_58k.collect()))

In [16]:
# Get user ids and ratings for subset of 58K books
books_58k = books_rdd.filter(lambda x: x[0] in broadcasted_book_ids.value)

In [17]:
books_58k.count()

1182213

After successfully filtering, the dataset has been reduced to just over 1M rows, and was downsampled based on the book IDs as opposed to the user IDs. 

In [18]:
# Convert first to Spark DF
books_58k_df = books_58k.toDF()

# Then convert to Pandas DF from use with the Surprise library
books_58k_df = books_58k_df.toPandas()

In [19]:
# Confirm data structure
books_58k_df.iloc[:2]

,_1,_2,_3,_4
0,2008343,A1RB3KF8ZQ43DZ,4.0,1203811200
1,2008343,A20QI7NG8SFN05,3.0,1199750400


### Build Recommender System with Python's Surprise Library

The data is returned in the format `(bookID, userID, rating, timestamp)`. 
To parse into a `Surprise` dataframe, it is expected to be in the format `(userID, bookID, rating, timestamp)`.

In [20]:
# Provide column headings to dataframe
books = books_58k_df.set_axis(['bookID', 'userID', 'rating', 'timestamp'], axis=1)

In [21]:
# Subset just user ID, book ID and ratings (out of 5 stars)
books = books[['userID', 'bookID', 'rating', 'timestamp']]
books.head(5)

,userID,bookID,rating,timestamp
0,A1RB3KF8ZQ43DZ,2008343,4.0,1203811200
1,A20QI7NG8SFN05,2008343,3.0,1199750400
2,A3V9X42L3AI67I,1472933,5.0,1360627200
3,AG7E2YAPMFJVL,1472933,3.0,1358640000
4,ARFK2WYVYT1QQ,1472933,5.0,1347321600


Now the pandas dataframe is in an acceptable format to be used by Suprise's algorithms.

In [22]:
# Import data into a Surprise DF
reader = Reader(rating_scale=(1.0, 5.0))
data = Dataset.load_from_df(books[['userID', 'bookID', 'rating']], reader=reader)

### Baseline SVD model

In [23]:
# Perform train test split
trainset, testset = train_test_split(data, test_size=0.2)

In [24]:
# Define baseline SVD 
model = SVD()

# Train and predict
model.fit(trainset)
preds = model.test(testset)

# Evaluate model
accuracy.rmse(preds)

RMSE: 0.9855


0.9854549930586427

Results from the baseline SVD model give an RMSE value of 0.989, or roughly an error of 1 full star-rating on average, on a 1-5 star rating scale. While this is not a terrible score, it can be improved upon for more accurate results.

### Perform Cross-validation to Randomize Splits

In [25]:
# Run 5-fold cross-validation and print results
cross_validate(model, data, measures=["RMSE", "MAE"], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9899  0.9868  0.9877  0.9876  0.9889  0.9882  0.0011  
MAE (testset)     0.7257  0.7227  0.7234  0.7237  0.7242  0.7239  0.0010  
Fit time          12.49   10.43   10.72   10.53   10.41   10.92   0.80    
Test time         0.74    0.75    0.76    1.38    1.38    1.00    0.31    


{'test_rmse': array([0.98988912, 0.98677177, 0.98768439, 0.9875813 , 0.98893956]),
 'test_mae': array([0.72571196, 0.72268489, 0.72341569, 0.72367834, 0.72423129]),
 'fit_time': (12.494518995285034,
  10.432440996170044,
  10.722224950790405,
  10.526863813400269,
  10.412353992462158),
 'test_time': (0.7356030941009521,
  0.7517130374908447,
  0.7563676834106445,
  1.3817241191864014,
  1.3761780261993408)}

Even with a simple cross-validation to rule out randomness in train/test split, the average RMSE score was still ~0.99.

### Perform GridSearch CV to find Best Params

In [26]:
# Define parameter grid
param_grid = {
    "n_epochs": [5, 10, 15, 20], 
    "lr_all": [0.002, 0.005, 0.01, 0.1], 
    "reg_all": [0.1, 0.4, 0.6]}
gs = GridSearchCV(SVD, param_grid, measures=["rmse", "mae"], cv=5)

# fit train data
gs.fit(data)

# best RMSE score
print(gs.best_score["rmse"])

# combination of parameters that gave the best RMSE score
print(gs.best_params["rmse"])

0.9837273688883142
{'n_epochs': 20, 'lr_all': 0.01, 'reg_all': 0.1}


In [27]:
# Run best model and perform cross-validation
tuned_model = SVD(n_epochs=20, lr_all=0.01, reg_all=0.1)

# Run 5-fold cross-validation and print results
cross_validate(tuned_model, data, measures=["RMSE", "MAE"], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9837  0.9837  0.9848  0.9821  0.9846  0.9838  0.0010  
MAE (testset)     0.7194  0.7205  0.7199  0.7200  0.7211  0.7202  0.0006  
Fit time          13.37   11.99   11.00   10.57   12.30   11.85   0.99    
Test time         1.29    0.75    0.76    0.76    0.77    0.87    0.21    


{'test_rmse': array([0.98368286, 0.98374204, 0.98476931, 0.98206963, 0.98459273]),
 'test_mae': array([0.71938808, 0.72050783, 0.71986622, 0.72001344, 0.72113289]),
 'fit_time': (13.3734769821167,
  11.986736059188843,
  11.001637935638428,
  10.574107885360718,
  12.299041032791138),
 'test_time': (1.2938790321350098,
  0.7538900375366211,
  0.7587981224060059,
  0.7577779293060303,
  0.7703983783721924)}

After hyperparameter tuning using GridSearch, the score improved only slightly to ~0.98 on average, after cross-validation.

### Try Neighbors-based Algorithms

In [24]:
# # Configure sim options
# sim_options = {
#     'name': 'msd',
#     'min_support': 10,
#     'user_based': False
# }

In [25]:
# # Baseline KNNBasic model
# knn = KNNWithMeans(verbose=True, sim_options=sim_options)

# # Fit and predict
# knn.fit(trainset=trainset)

# knn_preds = knn.test(testset=testset)

# # Evaluate model 
# accuracy.rmse(knn_preds)

: 